The script calculates the average time needed to purchase from the moment the product was added to the cart.

In [ ]:
%%bigquery 

with addToCart as (
select 
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date, 
    device.deviceCategory device,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    fullvisitorid,
    visitid,
    case when (SELECT value FROM hits.customDimensions WHERE index=51) in ('Zalogowany') then true else false end status, 
    hits.eCommerceAction.action_type, 
    min(hits.time) add_to_basket
FROM `empik-ga360.304756.ga_sessions_*` t, unnest(hits) as hits, unnest(product) as product
WHERE
    -- _table_suffix between '20200101' and '20211214'
    _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
and 
    (hits.eCommerceAction.action_type='3')
group by 
    1,2,3,4,5,6,7,8
having 
    add_to_basket > 0
),

bought as (
select 
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) as true_date, 
    device.deviceCategory device,
    format_date('%Y%U',parse_date("%Y%m%d",date)) as week_of_year,
    format_date('%Y%m',parse_date("%Y%m%d",date)) as month_of_year,
    fullvisitorid,
    visitid,
    case when (SELECT value FROM hits.customDimensions WHERE index=51) in ('Zalogowany') then true else false end status, 
    hits.eCommerceAction.action_type,
    min(hits.time) bought
FROM `empik-ga360.304756.ga_sessions_*` t, unnest(hits) as hits, unnest(product) as product
WHERE
    -- _table_suffix between '20200101' and '20211214'
    _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
    and (hits.eCommerceAction.action_type='6')
group by 
    1,2,3,4,5,6,7,8
having 
    bought > 0
)

select 
    true_date,
    week_of_year,
    month_of_year,
    device,
    status,
    avg(roznica) avg_roznica,

from (
select a.true_date, a.week_of_year, a.month_of_year, case when a.device like 'mobile' then 'mobile' else 'desktop' end device, a.status, a.add_to_basket, b.bought, (b.bought-a.add_to_basket)/1000 roznica
from addToCart a left join bought b on a.fullvisitorid=b.fullvisitorid and a.visitid=b.visitid and a.device=b.device and a.status=b.status
where bought > 0
)
group by 1,2,3,4,5
order by 1 desc